In [5]:
import os
import base64
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List,Optional
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from utils.img64 import read_image_base64
from langchain_core.messages import HumanMessage, SystemMessage
import mimetypes

In [4]:
os.chdir("D:\\Projects\\AI_projects\\Invoice OCR Extraction")

## Import Libraries

In [12]:
os.getcwd()

'D:\\Projects\\AI_projects\\Invoice OCR Extraction'

In [6]:
load_dotenv()

True

## Load Model

In [7]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0
)

In [8]:
def read_image_base64(img_path):
    with open(img_path, "rb") as img_data:
        img_base64 = base64.b64encode(img_data.read()).decode("utf-8")
    return img_base64

In [9]:
class InvoiceItem(BaseModel):
    description: str = Field(..., description="Item's Description")
    quantite: int = Field(..., description="Item's Quantite")
    prix: float = Field(..., description="Item's Prix")
    taxes: Optional[float] = Field(..., description="Item's Taxes")
    sous_total: int = Field(..., description="Item's Sous Total")
    
    

class InvoiceData(BaseModel):
    buyer_name: str = Field(..., description="Buyer's name in the invoice")
    invoice_date: str = Field(..., description="Buyer's name in the invoice")
    items: List[InvoiceItem] = Field(..., description="Item's data in the invoice")
    
parser = PydanticOutputParser(pydantic_object=InvoiceData)

In [10]:
examples = [
    {
        "invoice_img": read_image_base64(r"data\Invoices\2.jpg"),
        "invoice_data": InvoiceData(
                            buyer_name="Deco Addict",
                            invoice_date="04-04-2015",
                            items=[
                                InvoiceItem(
                                    description="Restaurant",
                                    quantite=23,
                                    prix=8,
                                    taxes=0.2,
                                    sous_total=184
                                ),
                                InvoiceItem(
                                    description="Architecte Principal (Facturation sur Feuilles de Temps)",
                                    quantite=49,
                                    prix=150,
                                    taxes=0.2,
                                    sous_total=7350
                                ),
                                InvoiceItem(
                                    description="Armoire avec portes",
                                    quantite=82,
                                    prix=120.50,
                                    taxes=0.1,
                                    sous_total=9881
                                ),
                            ]
                        ).model_dump_json()

    },
]

In [11]:
few_shot_examples_prompt = ChatPromptTemplate.from_messages(
    [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source_type": "base64",
                    "data": "{invoice_img}",
                    "mime_type": "image/jpeg",
                },
            ],
        },
        (
            "assistant","{invoice_data}"
        ),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = few_shot_examples_prompt,
    examples = examples
)

if __name__ == "__main__":
    for msg in few_shot_prompt.invoke({}).to_messages():
        msg.pretty_print()

================================ Human Message =================================

[{'type': 'image', 'source_type': 'base64', 'data': '...', 'mime_type': 'image/jpeg'}]
================================== Ai Message ==================================

{"buyer_name":"Deco Addict","invoice_date":"04-04-2015","items":[{"description":"Restaurant","quantite":23,"prix":8.0,"taxes":0.2,"sous_total":184},{"description":"Architecte Principal (Facturation sur Feuilles de Temps)","quantite":49,"prix":150.0,"taxes":0.2,"sous_total":7350},{"description":"Armoire avec portes","quantite":82,"prix":120.5,"taxes":0.1,"sous_total":9881}]}


In [12]:
msgsTemplate = ChatPromptTemplate.from_messages([ 
    ("system", """
     Extract data from this invoice
     follow this output schema {output_schema}
     """),
    few_shot_prompt,
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "source_type": "base64",
                "data": "{image_data}",
                "mime_type": "image/jpeg",
            },
        ],
    },
]).partial(output_schema=parser.get_format_instructions())

In [13]:
pipeline = msgsTemplate | llm | parser

In [14]:
def get_invoice_data(img_path):
    output = pipeline.invoke({
        "image_data": read_image_base64(img_path)
    })
    return output

if __name__ == "__main__":
    get_invoice_data(r"data\Invoices\3.jpg")

In [15]:
get_invoice_data(r"data\Invoices\3.jpg")

InvoiceData(buyer_name='The Jackson Group', invoice_date='05-07-2015', items=[InvoiceItem(description='Bureau personnalisable', quantite=49, prix=500.0, taxes=0.2, sous_total=24500), InvoiceItem(description="Design d'intérieur virtuel", quantite=99, prix=20.5, taxes=0.055, sous_total=2029), InvoiceItem(description="Bureau d'angle avec siège à gauche", quantite=79, prix=78.0, taxes=0.2, sous_total=6162), InvoiceItem(description='Grande table de réunion', quantite=34, prix=4500.0, taxes=0.2, sous_total=153000), InvoiceItem(description='Lampe de bureau', quantite=44, prix=35.0, taxes=0.2, sous_total=1540), InvoiceItem(description='Restaurant', quantite=16, prix=8.0, taxes=0.2, sous_total=128), InvoiceItem(description='Dépôt', quantite=87, prix=100.0, taxes=0.055, sous_total=8700), InvoiceItem(description='Bureau personnalisable', quantite=62, prix=500.0, taxes=0.2, sous_total=31000), InvoiceItem(description='Bureau pour quatre personnes', quantite=92, prix=2500.0, taxes=0.2, sous_total=23